In [ ]:
import json

In [ ]:
!pip install python-telegram-bot 

In [ ]:
!pip install fasttext
import fasttext
model = fasttext.load_model("/content/drive/MyDrive/Диплом/Модели/Модельw2v.bin")

Проверка работы(можно удалить)

Проверка


In [ ]:
import logging
from telegram import Update, ForceReply, ReplyKeyboardMarkup, ReplyKeyboardRemove, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext, CallbackQueryHandler,  ConversationHandler
# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)
global s,h,a,o
a=0
h=0
o=0
s=0
logger = logging.getLogger(__name__)


In [ ]:
def echo(update: Update, _: CallbackContext) -> None:
    """Echo the user message."""
    question = update.message.text
    if question=="😄": pred="__label__happy" 
    else: pred=model.predict(question)[0][0]
    print(update.message.text)
    reply = "Оценка: " + pred  +  "\nЕсли я ответил неправильно, ты можешь поправить меня. Научи меня, команда -  /teach_me" 
    update.message.reply_text(reply)

In [ ]:
FIRST, SECOND = range(2)
ONE, TWO, THREE, FOUR = range(4)
#ОЦЕНКА Да или Нет.  Если да, переходим на выход 3
#если нет, то переходим к оценке 
def teach_me(update: Update, context: CallbackContext) -> int:
    user = update.message.from_user
    logger.info("User %s started the conversation.", user.first_name)
    
    keyboard = [
        [
            InlineKeyboardButton("Да", callback_data=str(THREE)),
            InlineKeyboardButton("Нет", callback_data=str(ONE)),
        ]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)

    update.message.reply_text("Я ответил правильно? ", reply_markup=reply_markup)

    return FIRST




def one(update: Update, context: CallbackContext) -> int:
    """Show new choice of buttons"""
    query = update.callback_query
    query.answer()
    keyboard = [
        [
            InlineKeyboardButton("Happy", callback_data=str(THREE)),
            InlineKeyboardButton("Sad", callback_data=str(THREE)),
            InlineKeyboardButton("Angry", callback_data=str(THREE)),
            InlineKeyboardButton("Other", callback_data=str(THREE)),
        ]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    query.edit_message_text(
        text="Выбери правильный", reply_markup=reply_markup
    )
    return FIRST 
       
def three(update: Update, context: CallbackContext) -> int:
    """Show new choice of buttons"""
    query = update.callback_query
    query.answer()
    keyboard = [
        [
            InlineKeyboardButton("Статистика", callback_data=str(ONE)),
            InlineKeyboardButton("Нет спасибо", callback_data=str(TWO)),
        ]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    query.edit_message_text(
        text="Спасибо, что оценил. Я могу вывести статистику твоих сообщений. Как тебе идея?", reply_markup=reply_markup
    )
    # Transfer to conversation state `SECOND`
    return SECOND
    
def two(update: Update, context: CallbackContext) -> int:
    """Show new choice of buttons"""
    query = update.callback_query
    query.answer()
    keyboard = [
        [
            InlineKeyboardButton("1", callback_data=str(ONE)),
            InlineKeyboardButton("3", callback_data=str(THREE)),
        ]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    query.edit_message_text(
        text="Статистика\nHappy: 3\nAngry: 3\nSad: 1\nOthers: 0\nОбщее количество: 7")
    return FIRST
    
def stat(update: Update, context: CallbackContext) -> None:
    text="Статистика\nHappy: 3\nAngry: 1\nSad: 1\nOthers: 0\nОбщее количество: 5"
    update.message.reply_text(text)
    
def end(update: Update, context: CallbackContext) -> int:
    query = update.callback_query
    query.answer()
    query.edit_message_text(text="Ну как? Продолжим? Введи следующее сообщение")
    return ConversationHandler.END

def main() -> None:
    """Run the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater("1805287710:AAEuyNznhuueZSVz8ZKKjrLiKcKfTxCYF4A")

    dispatcher = updater.dispatcher

    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('teach_me', teach_me)],
        states={
            FIRST: [
                CallbackQueryHandler(one, pattern='^' + str(ONE) + '$'),
                CallbackQueryHandler(two, pattern='^' + str(TWO) + '$'),
                CallbackQueryHandler(three, pattern='^' + str(THREE) + '$'),
            ],
            SECOND: [
                CallbackQueryHandler(two, pattern='^' + str(ONE) + '$'),
                CallbackQueryHandler(end, pattern='^' + str(TWO) + '$'),
            ],
        },
        fallbacks=[CommandHandler('teach_me', teach_me)],
    )

    # Add ConversationHandler to dispatcher that will be used for handling updates
    dispatcher.add_handler(conv_handler)
    dispatcher.add_handler(CommandHandler("stat", stat))
    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()




In [ ]:
main()

/usr/local/lib/python3.7/dist-packages/telegram/ext/conversationhandler.py:288: UserWarning: If 'per_message=False', 'CallbackQueryHandler' will not be tracked for every message.
  "If 'per_message=False', 'CallbackQueryHandler' will not be "
2021-06-17 10:52:51,874 - apscheduler.scheduler - INFO - Scheduler started


thanks for a good job
Please dont chat with me again


2021-06-17 10:53:50,212 - __main__ - INFO - User Анастасия started the conversation.


I had a amazing day at sea today. Thanks for the advice
😄
thanks for a good job
thanks for a good job
Please dont chat with me again


2021-06-17 10:56:14,283 - __main__ - INFO - User Анастасия started the conversation.


I had a amazing day at sea today. Thanks for the advice
Recently I broke up with boyfriend. It hurts me
😄
I wanted you to answer my question. What are you writing? How stupid you are
A ok by wow you are :o) i don't want to talk to u


2021-06-17 10:56:56,926 - __main__ - INFO - User Анастасия started the conversation.
2021-06-17 11:38:27,559 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
2021-06-17 11:38:27,565 - apscheduler.scheduler - INFO - Scheduler has been shut down


In [ ]:
thanks for a good job

In [ ]:
I m in trouble

In [ ]:
I am satisfied with the result. I double-checked everything, everything is fine

In [ ]:
I had a amazing day at sea today. Thanks for the advice

In [ ]:
Please dont chat with me again

In [ ]:
I wanted you to answer my question. What are you writing? How stupid you are

In [ ]:
Recently I broke up with boyfriend. It hurts me

In [ ]:
My order didn't arrive on time. how did it happen

In [ ]:
ok by wow you are :o) i don't want to talk to u

In [ ]:
thanks for a good job

In [ ]:
thanks for a good job

In [ ]:
Please dont chat with me again 

In [ ]:
Неверное решение исправь

In [ ]:
I had a amazing day at sea today. Thanks for the advice

In [ ]:
Recently I broke up with boyfriend. It hurts me

In [ ]:
/stat

In [ ]:
I wanted you to answer my question. What are you writing? How stupid you are

In [ ]:
A ok by wow you are :o) i don't want to talk to u

In [ ]:
Вывести статистику